In [3]:
import pandas as pd
from fbprophet import Prophet # Prophet modelling library
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
   
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm



Importing plotly failed. Interactive plots will not work.


In [4]:
df = pd.read_csv('./Data/Data_1.csv')

In [5]:
#filter out quantity that is negative values as it will be irrelevant to our analysis
cdf = df[(df['Qty in Sales unit of measure (SUM)'])>0]
cdf

,Billing Date Invoiced Date,Country,Customer region redefined,Pr CFN,Pr Division,PrMPG Desc,Sold to Customer,Invoice Amount LOC (SUM),Qty in Sales unit of measure (SUM)
0,30/03/2021,Australia,Victoria,EASYCAPII6,MITG,Airway,120528591371,80.0,1.0
1,17/05/2021,New Zealand,New Zealand,NV-4-10-HELIX,Vascular,EV IR Channel,120007828721,800.0,1.0
2,28/05/2020,Australia,Victoria,RONYX25022X,Vascular,CORONARY,141125334293,710.0,1.0
3,29/04/2021,Australia,Queensland,PCO6VP,MITG,Synthetic Mesh,120471645278,407.0,1.0
4,11/02/2021,Australia,Northern Territory,LA6EBU35,Vascular,CORONARY,120028727566,340.0,4.0
...,...,...,...,...,...,...,...,...,...
1248668,29/09/2021,Australia,New South Wales,1883519HR,SurgicalTechnologies,XOMED ENT,202396479598,501.0,1.0
1248669,20/01/2022,Australia,New South Wales,1884004HR,SurgicalTechnologies,XOMED ENT,202396479598,882.0,1.0
1248670,08/10/2021,Australia,New South Wales,1898430,SurgicalTechnologies,XOMED ENT,202396479598,5008.5,1.0
1248671,11/08/2021,Australia,New South Wales,1425008,SurgicalTechnologies,XOMED ENT,203852017741,220.0,1.0


In [6]:
cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1215309 entries, 0 to 1248672
Data columns (total 9 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   Billing Date Invoiced Date          1215309 non-null  object 
 1   Country                             1215309 non-null  object 
 2   Customer region redefined           1215309 non-null  object 
 3   Pr CFN                              1215309 non-null  object 
 4   Pr Division                         1215309 non-null  object 
 5   PrMPG Desc                          1215309 non-null  object 
 6   Sold to Customer                    1215309 non-null  int64  
 7   Invoice Amount LOC (SUM)            1215309 non-null  float64
 8   Qty in Sales unit of measure (SUM)  1215309 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 92.7+ MB


In [7]:
cdf.dtypes

Billing Date Invoiced Date             object
Country                                object
Customer region redefined              object
Pr CFN                                 object
Pr Division                            object
PrMPG Desc                             object
Sold to Customer                        int64
Invoice Amount LOC (SUM)              float64
Qty in Sales unit of measure (SUM)    float64
dtype: object

In [8]:
cdf['Billing Date Invoiced Date'] = pd.to_datetime(
                          df['Billing Date Invoiced Date'],
                          format='%d/%m/%Y')
cdf['Sold to Customer'] = cdf['Sold to Customer'].astype(str)
cdf['year'] = cdf['Billing Date Invoiced Date'].dt.year
cdf['month'] = cdf['Billing Date Invoiced Date'].dt.month
cdf.dtypes

<ipython-input-8-6a2727fbde09>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf['Billing Date Invoiced Date'] = pd.to_datetime(
<ipython-input-8-6a2727fbde09>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf['Sold to Customer'] = cdf['Sold to Customer'].astype(str)
<ipython-input-8-6a2727fbde09>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

Billing Date Invoiced Date            datetime64[ns]
Country                                       object
Customer region redefined                     object
Pr CFN                                        object
Pr Division                                   object
PrMPG Desc                                    object
Sold to Customer                              object
Invoice Amount LOC (SUM)                     float64
Qty in Sales unit of measure (SUM)           float64
year                                           int64
month                                          int64
dtype: object

In [9]:
cdf

,Billing Date Invoiced Date,Country,Customer region redefined,Pr CFN,Pr Division,PrMPG Desc,Sold to Customer,Invoice Amount LOC (SUM),Qty in Sales unit of measure (SUM),year,month
0,2021-03-30,Australia,Victoria,EASYCAPII6,MITG,Airway,120528591371,80.0,1.0,2021,3
1,2021-05-17,New Zealand,New Zealand,NV-4-10-HELIX,Vascular,EV IR Channel,120007828721,800.0,1.0,2021,5
2,2020-05-28,Australia,Victoria,RONYX25022X,Vascular,CORONARY,141125334293,710.0,1.0,2020,5
3,2021-04-29,Australia,Queensland,PCO6VP,MITG,Synthetic Mesh,120471645278,407.0,1.0,2021,4
4,2021-02-11,Australia,Northern Territory,LA6EBU35,Vascular,CORONARY,120028727566,340.0,4.0,2021,2
...,...,...,...,...,...,...,...,...,...,...,...
1248668,2021-09-29,Australia,New South Wales,1883519HR,SurgicalTechnologies,XOMED ENT,202396479598,501.0,1.0,2021,9
1248669,2022-01-20,Australia,New South Wales,1884004HR,SurgicalTechnologies,XOMED ENT,202396479598,882.0,1.0,2022,1
1248670,2021-10-08,Australia,New South Wales,1898430,SurgicalTechnologies,XOMED ENT,202396479598,5008.5,1.0,2021,10
1248671,2021-08-11,Australia,New South Wales,1425008,SurgicalTechnologies,XOMED ENT,203852017741,220.0,1.0,2021,8


## Baseline model - Linear regression

In [10]:
#before modeling,converting data to be encoded.
le = LabelEncoder()
columns = ['Country','Customer region redefined','Pr Division','PrMPG Desc','Sold to Customer','Pr CFN']
for col in columns:
    cdf[col] = le.fit_transform(cdf[col])
cdf



<ipython-input-10-8cf8e29c4292>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf[col] = le.fit_transform(cdf[col])


,Billing Date Invoiced Date,Country,Customer region redefined,Pr CFN,Pr Division,PrMPG Desc,Sold to Customer,Invoice Amount LOC (SUM),Qty in Sales unit of measure (SUM),year,month
0,2021-03-30,0,8,7118,1,7,1461,80.0,1.0,2021,3
1,2021-05-17,1,2,8735,9,34,301,800.0,1.0,2021,5
2,2020-05-28,0,8,9417,9,21,5263,710.0,1.0,2020,5
3,2021-04-29,0,5,8971,1,96,1283,407.0,1.0,2021,4
4,2021-02-11,0,3,7921,9,21,321,340.0,4.0,2021,2
...,...,...,...,...,...,...,...,...,...,...,...
1248668,2021-09-29,0,1,1161,7,106,12072,501.0,1.0,2021,9
1248669,2022-01-20,0,1,1171,7,106,12072,882.0,1.0,2022,1
1248670,2021-10-08,0,1,1222,7,106,12072,5008.5,1.0,2021,10
1248671,2021-08-11,0,1,723,7,106,12322,220.0,1.0,2021,8


In [11]:
cdf.dtypes

Billing Date Invoiced Date            datetime64[ns]
Country                                        int32
Customer region redefined                      int32
Pr CFN                                         int32
Pr Division                                    int32
PrMPG Desc                                     int32
Sold to Customer                               int32
Invoice Amount LOC (SUM)                     float64
Qty in Sales unit of measure (SUM)           float64
year                                           int64
month                                          int64
dtype: object

In [12]:
X=cdf.drop(columns=['Qty in Sales unit of measure (SUM)','Billing Date Invoiced Date'])
y=cdf['Qty in Sales unit of measure (SUM)']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True, random_state=42, test_size=0.2)
print("Train split :", X_train.shape, y_train.shape)
print("Test split :", X_test.shape, y_test.shape)

Train split : (972247, 9) (972247,)
Test split : (243062, 9) (243062,)


In [14]:
lr = LinearRegression(normalize=True)
lr.fit(X_train,y_train)

print(f'R2 score for Baseline Model_Train: {round(lr.score(X_train,y_train),2)}')
print(f'R2 score for Baseline Model_Test: {round(lr.score(X_test,y_test),2)}')

C:\Users\ivan.cho\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


R2 score for Baseline Model_Train: 0.07
R2 score for Baseline Model_Test: 0.08


Linear Regression model will act as a benchmark (8% accuracy) for selecting model to be deployed, any model's R2 score falls below the benchmark, will be rejected.

## Facebook Prophet - Forecasting Time Series

In [63]:
fb_df = df[(df['Qty in Sales unit of measure (SUM)'])>0]
fb_df['Billing Date Invoiced Date'] = pd.to_datetime(
                          fb_df['Billing Date Invoiced Date'],
                          format='%d/%m/%Y')


<ipython-input-63-5b8f57f1d82d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_df['Billing Date Invoiced Date'] = pd.to_datetime(


## Predict sales trend in Product view

In [83]:
#Exclude unnecessary columns
fb_pv = fb_df[['Billing Date Invoiced Date','Pr CFN','Qty in Sales unit of measure (SUM)']]
#group together transaction that happened on the same day
fb_pv = fb_pv.groupby(['Billing Date Invoiced Date','Pr CFN']).sum().reset_index()


#filtering transactions that is more than two, because it is unreasonable to predict sales less than 5 transactions
v = fb_pv['Pr CFN'].value_counts()
fb_pv = fb_pv[fb_pv['Pr CFN'].isin(v.index[v.gt(5)])]
#get the unique material id
pv = fb_pv['Pr CFN'].unique()

In [84]:
fb_pv.isnull().sum()

Billing Date Invoiced Date            0
Pr CFN                                0
Qty in Sales unit of measure (SUM)    0
dtype: int64

In [85]:
#make sure there are more than one transactions for every materials
fb_pv['Pr CFN'].value_counts()

8886803712     458
W2DR01         458
5076-52        458
EGIA60AMT      457
E1455          457
              ... 
88863222-56      6
1081275          6
SCD48            6
88861742-31      6
DXT6JL40X        6
Name: Pr CFN, Length: 7025, dtype: int64

In [ ]:
predict = pd.DataFrame()
for prod in pv:
    frame_pv = fb_pv[fb_pv['Pr CFN']==prod].copy()
    frame_pv.drop('Pr CFN', axis=1,inplace=True)
    frame_pv.columns = ['ds', 'y']
    
    #using Full bayesian sampling to anticipate the uncertainty of the trend
    print(prod)
    mpv = Prophet(mcmc_samples=300)
    model = mpv.fit(frame_pv)
    #create empty dictionary to store the model
    pv_model = {}
    pv_model[prod] = mpv
    #making prediction for 24 months
    forward = pv_model[prod].make_future_dataframe(periods=24, freq='M')
    forecast = pv_model[prod].predict(forward)
    #store prediction into a dataframe
    pred_dict = pd.DataFrame({'material':prod,'date':forecast['ds'],'predict':forecast['yhat']})
    #append it into the main dataframe together with all other materials
    predict.append(pred_dict)

predict

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


007042


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


007046


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

118-80MTG


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


118-85M


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 19.


134044


### Computing R-Square and Mean Square error

In [ ]:
eval_pred = pd.merge(predict,fb_pv, left_on=['date','material'], right_on=['Pr CFN','Billing Date Invoiced Date'], how='left')
eval_pred = eval_pred.dropna()
r2_score(eval_pred['Qty in Sales unit of measure (SUM)'],eval_pred.yhat)

In [ ]:
predict.to_csv('./Data/FBprophet prediction.csv')

## ARIMA - Time Series Forecast

In [ ]:
#filter out less than zero quantity
arima = df[(df['Qty in Sales unit of measure (SUM)'])>0]
#remove unneccessary columns
arima = arima[['Billing Date Invoiced Date','Pr CFN','Qty in Sales unit of measure (SUM)']]
#convert invoiced date to datetime format
arima['Billing Date Invoiced Date'] = pd.to_datetime(
                          arima['Billing Date Invoiced Date'],
                          format='%d/%m/%Y')
#Add new column for year
arima['year'] = arima['Billing Date Invoiced Date'].dt.year
#Add new column for month
arima['month'] = arima['Billing Date Invoiced Date'].dt.month
#create groupby transactions date for time series prediction
arima = arima.groupby(['year','month','Pr CFN']).sum().reset_index()
#create column to convert year and month to first date of the month
arima['date'] = pd.to_datetime(arima[['year', 'month']].assign(DAY=1))
v = arima['Pr CFN'].value_counts()
#filter out transactions less than 15 because ARIMA need more sufficient data point to make predictions.
arima = arima[arima['Pr CFN'].isin(v.index[v.gt(15)])]
#get the unique material id
pv = arima['Pr CFN'].unique()
arima

In [ ]:
arima['Pr CFN'].value_counts().sort_values(ascending=True)

In [ ]:
arima.isnull().sum()

In [ ]:
#Empty Dataframe for prediction
arima_pred = pd.DataFrame()
arima_merge = pd.DataFrame()
for prod_x in pv:
    frame_arima = arima[arima['Pr CFN']==prod_x].copy()
    frame_arima.drop(['Pr CFN','year','month'], axis=1,inplace=True)
    frame_arima.columns = [ 'y','date']
    print(prod_x)
    model = pm.auto_arima(frame_arima.y, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=10, max_q=3, # maximum p and q
                          m=12,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False,   # No Seasonality
                          start_P=0, 
                          D=0, 
                          alpha=0.1,
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)


    # Forecast + Test data
    n_periods = 34
    #prediction for the next 24 months + 10months of testing
    fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
    #create date range for 24 months + 10 months of testing
    index_of_fc = pd.date_range(frame_arima['date'].iloc[-10], periods = n_periods, freq='MS')
    #combine date with the predicted values
    fc_series = pd.DataFrame(fc, index=index_of_fc)
    #upper and lower confidence level of the prediction
    lower_series = pd.Series(confint[:, 0])
    upper_series = pd.Series(confint[:, 1])
    #append all into a dataframe
    arima_dict = pd.DataFrame({'y':fc,'date':index_of_fc,'material':prod_x,'lower':lower_series,'upper':upper_series})
    arima_df = pd.DataFrame({'actual':frame_arima['y'],'date':frame_arima['date'],'material':prod_x})
    arima_merge = arima_df.merge(arima_dict, left_on = 'date', right_on='date', how='outer')
    arima_pred = arima_pred.append(arima_merge)
    

In [ ]:
arima_pred.head(50)

In [ ]:
arima_pred.to_csv('./Data/ARIMA prediction.csv')


In [ ]:
eval_arima = arima_pred.dropna()
# eval_arima
print(f'R2 score for Arima is {round(r2_score(eval_arima["actual"],eval_arima.y),2)}')

**R-squared (R2)** is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.So R2 explains to an extent that the variance of one variable explains the variance of the second variable. With that, Our ARIMA model has **50%** R2, means that approximately half of the observed variation can be explained by the ARIMA's input.